In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import scipy as sp
import warnings
warnings.filterwarnings('ignore')

1. Does the size of sets changed over time?

In [75]:
df_sets = pd.read_csv("/kaggle/input/lego-database/sets.csv")
df_sets.head()

,set_num,name,year,theme_id,num_parts
0,00-1,Weetabix Castle,1970,414,471
1,0011-2,Town Mini-Figures,1978,84,12
2,0011-3,Castle 2 for 1 Bonus Offer,1987,199,2
3,0012-1,Space Mini-Figures,1979,143,12
4,0013-1,Space Mini-Figures,1979,143,12


In [116]:
# df_sets.sort_values(by='num_parts', ascending = False)['name'][0]
# df_sets.sort_values(by='num_parts', ascending = False)['num_parts'][0]
print('The larger LEGO set is', df_sets.loc[df_sets['num_parts'] == df_sets['num_parts'].max()]['name'],'with',df_sets['num_parts'].max(), 'parts.')
# df_sets.loc[df_sets['num_parts'] == df_sets['num_parts'].max()]['name']

The larger LEGO set is 170    Taj Mahal
Name: name, dtype: object with 5922 parts.


In [130]:
df_sets.name.loc[df_sets['num_parts'] == df_sets['num_parts'].max()].str.split()[2:]

Series([], Name: name, dtype: object)

In [121]:
df_sets.loc[df_sets['name'] == "Shark's Crystal Cave"]


,set_num,name,year,theme_id,num_parts
5922,6190-1,Shark's Crystal Cave,1996,311,259


In [85]:
df_sets.sort_values(by='num_parts', ascending = False)

,set_num,name,year,theme_id,num_parts
170,10189-1,Taj Mahal,2008,276,5922
11614,SWMP-1,Star Wars / M&M Mosaic - Promo Set,2005,169,5461
1337,2000409-1,Window Exploration Bag,2010,507,5200
161,10179-1,Millennium Falcon - UCS,2007,174,5195
8322,75827-1,Firehouse Headquarters,2016,301,4640
...,...,...,...,...,...
9639,852194-1,Dwarf Key Chain,2008,503,0
9723,853187-1,Captain Jack Sparrow Key Chain,2011,501,0
6545,66392-1,"Duplo Cars Super Pack 3 in 1 (5816, 5817, 5818)",2012,506,-1
11645,Vancouver-1,"LEGO Store Grand Opening Exclusive Set, Oakrid...",2012,408,-1


In [25]:
sets_size = df_sets.groupby('year').agg(Mean=('num_parts', np.mean),
                                        Std=('num_parts', np.std))

Fig = px.scatter(sets_size,
                 x = sets_size.index,
                 y = 'Mean',
                 height = 400,
                 width = 600,
#                 error_y= 'Std',
                 labels = {'year':'Year',
                         'num_parts' : "Average number of part "},
                 title = "Evolution of mean size of LEGO's set")
Fig.show()

r, p = sp.stats.pearsonr(sets_size.index, sets_size['Mean'])

if p < 0.05:
    print("The evolution of the size of LEGO's set over time is significant.")
else:
    print("There has not been significant changes in the size of LEGO's set over time.")

The evolution of the size of LEGO's set over time is significant.


In [27]:
df_themes = pd.read_csv("/kaggle/input/lego-database/themes.csv")
df_themes.head()

,id,name,parent_id
0,1,Technic,NaN
1,2,Arctic Technic,1.0
2,3,Competition,1.0
3,4,Expert Builder,1.0
4,5,Model,1.0


In [42]:
df_merged = pd.merge(df_sets, df_themes, left_on = 'theme_id', right_on = 'id')
df_merged.head()

,set_num,name_x,year,theme_id,num_parts,id,name_y,parent_id
0,00-1,Weetabix Castle,1970,414,471,414,Castle,411.0
1,0011-2,Town Mini-Figures,1978,84,12,84,Supplemental,67.0
2,0011-3,Castle 2 for 1 Bonus Offer,1987,199,2,199,Lion Knights,186.0
3,0012-1,Space Mini-Figures,1979,143,12,143,Supplemental,126.0
4,0013-1,Space Mini-Figures,1979,143,12,143,Supplemental,126.0


In [43]:
df_merged.drop(['theme_id', 'id', 'parent_id'], axis = 1, inplace = True)
df_merged.rename(columns = {'name_x' : 'set',
                            'name_y' : 'theme'}, inplace = True)
df_merged.head()

,set_num,set,year,num_parts,theme
0,00-1,Weetabix Castle,1970,471,Castle
1,0011-2,Town Mini-Figures,1978,12,Supplemental
2,0011-3,Castle 2 for 1 Bonus Offer,1987,2,Lion Knights
3,0012-1,Space Mini-Figures,1979,12,Supplemental
4,0013-1,Space Mini-Figures,1979,12,Supplemental


In [73]:
sets_size_theme = df_merged.groupby(['theme']).agg(Mean = ('num_parts' , 'mean'), Std = ('num_parts' , 'std')).sort_values(by = 'Mean', ascending = False)[:15]

Fig = px.scatter(sets_size_theme,
                 y = sets_size_theme.index,
                 x = sets_size_theme['Mean'],
                 error_x = 'Std',
                 color = sets_size_theme.index)

Fig.update_traces(marker=dict(size=20))

Fig.show()

In [55]:
sets_size_theme = df_merged.groupby(['theme']).agg({'num_parts' : 'mean'}).sort_values(by = 'num_parts', ascending = False)
sets_size_theme

,num_parts
theme,
Modular Buildings,2350.583333
Mosaic,1843.250000
Sculptures,1716.695652
Ultimate Collector Series,1680.333333
FIRST LEGO League,1387.000000
...,...
Power Functions,1.882353
Imperial Guards,1.000000
Key Chain,0.181818


In [ ]:
df_parts = pd.read_csv("/kaggle/input/lego-database/parts.csv")
df_parts.head()

In [ ]:
df_color = pd.read_csv("/kaggle/input/lego-database/colors.csv")
df_sets = pd.read_csv("/kaggle/input/lego-database/sets.csv")
df_themes = pd.read_csv("/kaggle/input/lego-database/themes.csv")
df_inventory_sets = pd.read_csv("/kaggle/input/lego-database/inventory_sets.csv")
df_inventories = pd.read_csv("/kaggle/input/lego-database/inventories.csv")
df_inventory_parts = pd.read_csv("/kaggle/input/lego-database/inventory_parts.csv")
df_parts = pd.read_csv("/kaggle/input/lego-database/parts.csv")
df_part_categories = pd.read_csv("/kaggle/input/lego-database/part_categories.csv")

In [ ]:
df_themes = pd.read_csv("/kaggle/input/lego-database/themes.csv")
df_themes.head()

In [ ]:
df_inventory_sets.head()

2. What colors are associated with which theme?